# MNIST Object detection using YOLOv4

In [5]:
import os

In [6]:
!git submodule update --init --recursive
%pip install -r MNIST-ObjectDetection/requirements.txt yolov4
if not os.path.exists('MNIST-ObjectDetection/data'):
    %cd MNIST-ObjectDetection
    !python generate_data.py
    %cd ..

Note: you may need to restart the kernel to use updated packages.


In [7]:
GPU_NAME =  str(os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read()).strip()
GPU_COMPUTE_CAPABILITY = str(os.popen('nvidia-smi --query-gpu=compute_cap --format=csv,noheader').read()).strip()
# Remove newlines
GPU_COMPUTE_CAPABILITY = GPU_COMPUTE_CAPABILITY.replace('.', '')

print(f'GPU_NAME: {GPU_NAME}')
print(f'GPU_COMPUTE_CAPABILITY: {GPU_COMPUTE_CAPABILITY}')
# Build using make, enable OpenCV and CUDNN
import re
with open('darknet/Makefile', 'r') as f:
    makefile = f.read()
makefile = re.sub(r'GPU=0', 'GPU=1', makefile)
makefile = re.sub(r'CUDNN=0', 'CUDNN=1', makefile)
makefile = re.sub(r'CUDNN_HALF=0', 'CUDNN_HALF=1', makefile)
makefile = re.sub(r'OPENCV=0', 'OPENCV=1', makefile)

# Remove everything in arch= and remove lines after it if it ends with a \. Replace with GPU_COMPUTE_CAPABILITY
makefile = re.sub(r'ARCH=(.*\\\n)*.*', f'ARCH={GPU_COMPUTE_CAPABILITY}', makefile)


# Enable CUDNN_HALF if GPU compute capability >= 75
if int(GPU_COMPUTE_CAPABILITY) >= 75:
    makefile = re.sub(r'CUDNN_HALF=0', 'CUDNN_HALF=1', makefile)

with open('darknet/Makefile', 'w') as f:
    f.write(makefile)

# Build darknet
!(cd darknet && make -j)

GPU_NAME: NVIDIA GeForce RTX 4070 Ti
GPU_COMPUTE_CAPABILITY: 89
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh


process_begin: CreateProcess(NULL, uname, ...) failed.
Makefile:23: pipe: No error
La syntaxe de la commande n'est pas correcte.
process_begin: CreateProcess(NULL, chmod +x *.sh, ...) failed.
make (e=2): Le fichier sp�cifi� est introuvable.

make: *** [Makefile:203: setchmod] Error 2
make: *** Waiting for unfinished jobs....
make: *** [Makefile:197: obj/] Error 1


In [8]:
# Use YOLOv4 to detect MNIST digits from images

# Structure is train -> images -> 0,1,2,3,4,5,6,7,8,9 -> 0.jpg, 1.jpg, 2.jpg, ...
# Structure is train -> labels -> 0,1,2,3,4,5,6,7,8,9 -> 0.txt, 1.txt, 2.txt, ...
# Same for test

data_path = 'MNIST-ObjectDetection/data/mnist_detection'
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')

# Train YOLOv4 on MNIST dataset